

# Source :
https://python.plainenglish.io/connect-xero-oauth-2-0-using-python-without-a-website-86c79474c482



Notes : I also tested, you can create two apps with different client ids , they will work in on the same xero data


### format of Trial Balance
https://developer.xero.com/documentation/api/accounting/reports/#budget-summary

### XERO API Exploration
https://api-explorer.xero.com/accounting/reports/getreporttrialbalance?query-date=2022-06-30

### Auth flow - secret code
https://developer.xero.com/documentation/guides/oauth2/auth-flow#4-receive-your-tokens


### Auth flow - PKCE
https://developer.xero.com/documentation/guides/oauth2/pkce-flow

### Create App - developer site
https://developer.xero.com/

In [272]:

client_id = '997B883FBD8F4100B021ABB5D83C5DE3' # web client id
client_secret = 'Bk90_tGJsnRY6LrkvzneoXPLajzU04Mq4EW0ub5-iS7G2UBV'
redirect_url = 'https://developer.xero.com/'
#scope = 'offline_access accounting.contacts'
scope = 'offline_access accounting.reports.read'
b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

# Generate authentication URL and connect to xero

In [273]:
auth_url = ('https://login.xero.com/identity/connect/authorize?' +
'response_type=code' +
'&client_id=' + client_id +
'&scope=' + scope +
'&redirect_uri=' + redirect_url)
print (auth_url)

https://login.xero.com/identity/connect/authorize?response_type=code&client_id=997B883FBD8F4100B021ABB5D83C5DE3&scope=offline_access accounting.reports.read&redirect_uri=https://developer.xero.com/


In [81]:
code = 'd5b699afb0e611a51a41e4633674adf7804e87b5bdba0e56c8f35ebc00bdae8e'

# Exchange token

In [84]:
 # 3. Exchange the code
exchange_code_url = 'https://identity.xero.com/connect/token'
response = requests.post(exchange_code_url, 
                        headers = {
                            'Authorization': 'Basic ' + b64_id_secret
                        },
                        data={
              'grant_type': 'authorization_code',
              'code': code,
              'redirect_uri': redirect_url
})

In [85]:
response.json()

{'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjFDQUY4RTY2NzcyRDZEQzAyOEQ2NzI2RkQwMjYxNTgxNTcwRUZDMTkiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJISy1PWm5jdGJjQW8xbkp2MENZVmdWY09fQmsifQ.eyJuYmYiOjE2NTI3OTIyMjcsImV4cCI6MTY1Mjc5NDAyNywiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS54ZXJvLmNvbSIsImF1ZCI6Imh0dHBzOi8vaWRlbnRpdHkueGVyby5jb20vcmVzb3VyY2VzIiwiY2xpZW50X2lkIjoiOTk3Qjg4M0ZCRDhGNDEwMEIwMjFBQkI1RDgzQzVERTMiLCJzdWIiOiI0Zjk3ZjQ4M2Q1MTI1NmFkOTg4ZDMzOTQ2YTdhMWEyOSIsImF1dGhfdGltZSI6MTY1Mjc4NjgyMiwieGVyb191c2VyaWQiOiIyMTc5ZWM0Ni03MjcxLTRiOTktYjY2ZC1jNWQ5NGExOGRjYjQiLCJnbG9iYWxfc2Vzc2lvbl9pZCI6IjMzMTc5MTJkY2U2YjQ1YzJiNGMzYjU0ZjJhZjAyOTdjIiwianRpIjoiY2JhNGYxNTgwYWZiOGFjZDI2MGUxYTRjZWE4NzE2MzUiLCJhdXRoZW50aWNhdGlvbl9ldmVudF9pZCI6ImQ0MTNmNGUwLWI2NjEtNDZkNC04YjhjLTk2MGFlZTU5NDc0ZCIsInNjb3BlIjpbImFjY291bnRpbmcucmVwb3J0cy5yZWFkIiwib2ZmbGluZV9hY2Nlc3MiXSwiYW1yIjpbInB3ZCJdfQ.nxNNzpBBNSVxAn78Jzv5kaz2Tlh_pGgHBPDmwzNyt7QHJuSD-7o2P7tPS9uu7n-hjd8xN5TW7oYMurLHlt0ykAuvZKgjnLKPsyWzf7k5F3vJvxJpXYgksUwpYkqH3ke3-dqfMn-w5vb7Z1MwZ_gHcZxdRsQL

In [87]:
access_token = response.json()['access_token']
refresh_token = response.json()['refresh_token']

# store the tenant id in list for looping

In [20]:
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
             'Authorization' : 'Bearer '+access_token,
             'Content-Type' : 'application/json'
          })
conn_json = response.json()
mylist = []
tenantName = ""
for t in conn_json:
    if 'tenantId' in t:
        tenantId = t['tenantId']
        mylist.append(tenantId)

In [24]:
mylist

['6439291f-b6b3-433a-b6b3-810336a7c903',
 'c2635a39-2e2a-4105-8aa9-b4800c4f59dc']

In [89]:
import json
#url = 'https://api.xero.com/api.xro/2.0/Contacts'
url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance'
#url = 'https://api.xero.com/connections'

for i in mylist:
    response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json',
               'Xero-tenant-id' : i})
    data = json.loads(response.text)
    with open(i+'.txt','w') as f:
        f.write(str(data))
        #### it works i can write different text files.
        ### just need to figure out tidying up the text to proper columns etc like csv

In [275]:
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
json.loads(response.text)
  

[{'id': 'dea2dd9c-9366-450e-8964-dd18e5814abc',
  'authEventId': '9dbcdca8-1f38-4caa-84fb-01870c08eed8',
  'tenantId': '6439291f-b6b3-433a-b6b3-810336a7c903',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Xero Testing2',
  'createdDateUtc': '2022-05-17T06:31:48.1219710',
  'updatedDateUtc': '2022-05-17T06:31:48.1231110'},
 {'id': '89e470f4-503a-44dc-af06-63f68e6769b2',
  'authEventId': '0cc2e136-620d-4a9a-80ba-c1ddb4887fb4',
  'tenantId': 'c2635a39-2e2a-4105-8aa9-b4800c4f59dc',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Xero Testing',
  'createdDateUtc': '2022-05-17T06:31:12.4294250',
  'updatedDateUtc': '2022-05-17T06:31:12.4306880'},
 {'id': 'aa7e2f9a-1077-4b18-9dda-04e37c9d0004',
  'authEventId': 'a3f5710e-c948-4110-95e1-3e8a9a9a7468',
  'tenantId': 'ef778bbb-e6cb-4f92-8cfe-4e165c0da7ee',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Demo Company (AU)',
  'createdDateUtc': '2022-05-18T12:27:36.6094690',
  'updatedDateUtc': '2022-05-18T12:27:36.6111980'}]

# Refresh Token

Please note refreshen token expires after 60 days

In [274]:
import requests
import base64

b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')


# open last refresh token
f = open('oath_refresh.txt','r')
refresh_token = f.read()


# get new access token and refresh token
url = 'https://identity.xero.com/connect/token'
response = requests.post(url,
           headers={
               'Authorization': 'Basic ' + b64_id_secret
           },
           data={
               'grant_type': 'refresh_token',
               'client_id' : client_id,
               'refresh_token': refresh_token
           })


response.json()
response_dict = response.json()
access_token = response_dict['access_token']
refresh_token = response_dict['refresh_token']


# save new refresh token
f = open('oath_refresh.txt','w')
f = f.write(refresh_token)

# Convert Json to DF

In [278]:
import json
#url = 'https://api.xero.com/api.xro/2.0/Contacts'
#url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance'
url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance?date=2022-04-30&paymentsOnly=false'
#url = 'https://api.xero.com/connections'


response = requests.get(url,
       headers={
           'Authorization' : 'Bearer '+access_token,
           'Accept' : 'application/json',
           'Xero-tenant-id' : 'ef778bbb-e6cb-4f92-8cfe-4e165c0da7ee'})
data = json.loads(response.text)


In [279]:


mycell = []
for i in data['Reports']:
    for j, l1 in enumerate(i['Rows']):
        #print (j,desc['RowType'],'\n')
        if j != 0:
            # go to rows section and loop it to find cells
            #print (j,desc,'\n')
            for l2 in l1['Rows']:
                #print (l2,'\n')
                myrow = []
                for l3 in l2['Cells']:
                  #print (l3['Value'])
                  myrow.append(l3['Value'])
                mycell.append(myrow)
        
df = pd.DataFrame(mycell)
df.columns = ['Desc','Dr','Cr','YTD Dr','YTD Cr']

df

,Desc,Dr,Cr,YTD Dr,YTD Cr
0,Sales (200),,13645.00,,35931.00
1,Advertising (400),1818.18,,1830.18,
2,Bank Fees (404),10.50,,31.50,
3,Cleaning (408),155.00,,310.00,
4,Consulting & Accounting (412),49.00,,49.00,
5,Entertainment (420),,200.00,277.20,
6,General Expenses (429),0.00,,966.87,
7,"Light, Power, Heating (445)",301.00,,946.00,
8,Motor Vehicle Expenses (449),59.09,,429.09,
9,Office Expenses (453),83.54,,1410.44,


# Simple example to convert json to csv

In [199]:
x = {'Name':'Ronny','Address':'Sydney'}

In [202]:
x.values()

dict_values(['Ronny', 'Sydney'])

In [194]:
data = {'emp_details':[{'Name':'Ronny','Address':'Sydney'},{'Name':'Mercedes','Address':'Melbourne'}]}
mystring = json.dumps(data)
mylist = json.loads(m)


data['emp_details']


TypeError: the JSON object must be str, bytes or bytearray, not dict

In [85]:
employee_data

[{'Name': 'Ronny', 'Address': 'Sydney'},
 {'Name': 'Mercedes', 'Address': 'Melbourne'}]

In [82]:
import json
import csv
 
 
employee_data = data['emp_details']
 
# now we will open a file for writing
data_file = open('data_file.csv', 'w')
 
# create the csv writer object
csv_writer = csv.writer(data_file)
 
# Counter variable used for writing
# headers to the CSV file
count = 0
 
for emp in employee_data:
    if count == 0:
 
        # Writing headers of CSV file
        header = emp.keys()
        csv_writer.writerow(header)
        count += 1
 
    # Writing data of CSV file
    csv_writer.writerow(emp.values())
 
data_file.close()

In [83]:
import pandas as pd

pd.read_csv('data_file.csv')

,Name,Address
0,Ronny,Sydney
1,Mercedes,Melbourne


In [6]:
employee_data = data['emp_details']
employee_data

[{'Name': 'Ronny', 'Address': 'Sydney'},
 {'Name': 'Mercedes', 'Address': 'Melbourne'}]

# test xero json to csv